In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import math
import scipy
from PIL import image
from scipy import ndimage
import tensorflow as tf
from tensorflow.python.framework import ops


plt.rcParams['figure.figsize'] = (5.0, 4.0)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

np.random.seed(1)

def zero_pad(X, pad):
    X_pad = np.pad(X,((0,0),(pad,pad),(pad,pad),(0,0)),'constant')
    return X_pad

def conv_single_step(a_slice_prev, W, b):
    '''
    a_slice_prev:slice of input data of shape(f,f,n_C_prev)
    W: shape (f,f,n_C_prev)
    b: shape (1,1,1)
    '''
    s = a_slice_prev * W
    Z = np.sum(s)
    Z = Z+b
    return Z

def conv_forward(A_prev, W, b, hparams):
    (m,n_H_prev,n_W_prev,n_C_prev) = A_prev.shape
    (f,f,n_c_prev,n_C) = W.shape
    stride = hparams['stride']
    pad = hparams['pad']
    
    n_H = int((n_H_prev+2*pad-f)/stride+1)
    n_W = int((n_W_prev+2*pad-f)/stride+1)
    
    Z = np.zeros((m,n_H,n_W,n_C))
    
    A_prev_pad = zero_pad(A_prev,pad)
    
    for i in range(m):
        a_prev_pad = A_prev_pad[i,:,:,:]
        for h in range(n_H):
            for w in range(n_W):
                for c in range(n_C):
                    
                    vert_start = stride*h
                    vert_end = vert_start+f
                    horiz_start = stride*w
                    horiz_end = horiz_start+f
                    
                    a_slice_prev = a_prev_pad[vert_start:vert_end,horiz_start:horiz_end,:]
                    Z[i,h,w,c] = conv_single_step(a_slice_prev,W[:,:,:,c],b[:,:,:,c])
    cache = (A_prev,W,b,hparams)
    return Z,cache

def pool_forward(A_prev,hparams,mode='max'):
    (m,n_H_prev,n_W_prev,n_C_prev) = A_prev.shape
    f = hparams['f']
    stride = hparams['stride']
    
    n_H = int((n_H_prev-f)/stride + 1)
    n_W = int((n_W_prev-f)/stride + 1)
    n_C = n_C_prev
    
    A = np.zeros((m,n_H,n_W,n_C))
    for i in range(m):
        for h in range(n_H):
            for w in range(n_W):
                for c in range(n_C):
                    
                    vert_start = h*stride
                    vert_end = vert_start + f
                    horiz_start = w*stride
                    horiz_end = horiz_start + f
                    
                    a_prev_slice = A_prev[i,vert_start:vert_end,horiz_start:horiz_end,c]
                    if mode == "max":
                        A[i,h,w,c] = np.max(a_prev_slice)
                    elif mode == 'average':
                        A[i,h,w,c] = np.mean(a_prev_slice)
    cache = (A_prev, hparams)
    return A, cache

def conv_backward(dZ, cache):
    (A_prev,W,b,hparams) = cache
    (m,n_H_prev,n_W_prev,n_C_prev) = A_prev.shape
    (f,f,n_C_prev,n_C) = W.shape
    
    stride = hparams['stride']
    pad = hparams['pad']
    
    (m,n_H,n_W,n_C) = dZ.shape
    
    dA_prev = np.zeros((m,n_H_prev,n_W_prev,n_C_prev))
    dW = np.zeros((f,f,n_C_prev,n_C))
    db = np.zeros((1,1,1,n_C))
    
    A_prev_pad = zero_pad(A_prev, pad)
    dA_prev_pad = zero_pad(dA_prev, pad)
    
    for i in range(m):
        a_prev_pad = A_prev_pad[i,:,:,:]
        da_prev_pad = dA_prev_pad[i,:,:,:]
        
        for h in range(n_H):
            for w in range(n_W):
                for c in range(n_C):
                    vert_start = h * stride
                    vert_end = vert_start + f
                    horiz_start = w * stride
                    horiz_end = horiz_start + f
                    
                    a_slice = a_prev_pad[vert_start:vert_end,horiz_start:horiz_end,:]
                    da_prev_pad[vert_start:vert_end,horiz_start:horiz_end,:] += W[:,:,:,C]*dZ[i,h,w,c]
                    dW[:,:,:,c] += a_slice * dZ[i,h,w,c]
                    db[:,:,:,c] += dZ[i,h,w,c]
        dA_prev[i,:,:,:] = da_prev_pad[pad:-pad,pad:-pad,:]
    return dA_prev, dW, db

def create_mask_from_window(x):
    mask = (x == np.max(x))
    return mask

def distribute_value(dz, shape):
    (n_H, n_W) = shape
    average = dz /(n_H*n_W)
    a = average * np.ones(shape)
    return a

def pool_backward(dA, cache, mode='max'):
    (A_prev, hparams) = cache
    stride = hparams["stride"]
    f = hparams['f']

    m,n_H_prev,n_W_prev,n_C_prev = A_prev.shape
    m,n_H,n_W,n_C = dA.shape

    dA_prev = np.zeros(np.shape(A_prev))
    for i in range(m):
        a_prev = A_prev[i,:,:,:]

        for h in range(n_H):
            for w in range(n_W):
                for c in range(n_C):
                    vert_start = h * stride
                    vert_end = vert_start + f
                    horiz_start = w * stride
                    horiz_end = horiz_start + f

                    if mode == 'max':
                        a_prev_slice = a_prev[vert_start:vert_end,horiz_start:horiz_end,c]
                        mask = create_mask_from_window(a_prev_slice)
                        dA_prev[i,vert_start:vert_end,horiz_start:horiz_end,c]+=np.multiply(mask,dA[i,h,w,c])
                    elif mode == 'average':
                        da = dA[i,h,w,c]
                        shape = (f,f)
                        dA_prev[i,vert_start:vert_end,horiz_start:horiz_end,c]+=distribute_value(da, shape)
    return dA_prev


X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()